In [77]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )
import json
import requests
import pandas as pd
from decimal import Decimal
from carbon.carbon_order_ui import CarbonOrderUI
from carbon.sdk.carbonsdk import EncodedOrder
from benchmark import alphaxutils

Carbon Version v2.4.1-BETA1 (17/Mar/2023)


# Carbon Simulation - Demo TestSuite

This book is for creating test scenarios for Tenderly Contracts

#### Basic token getter for decimals

In [2]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = 18
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = int(cgtokens.decimals[i])

## Enter tkn0 and tkn1

In [3]:
tkn0 = "ETH"  #base token
tkn1 = "USDC" #quote token

token0Decimals = tokenDecimals[tkn0]
token1Decimals = tokenDecimals[tkn1]
delta_tokenDecimals = token1Decimals - token0Decimals
print(tkn0, token0Decimals)
print(tkn1, token1Decimals)

ETH 18
USDC 6


## Initialize Simulator instance

In [56]:
Sim = CarbonSimulatorUI(pair=f"{tkn0}/{tkn1}", verbose=False, matching_method='fast')  # important to denote fast matching. pairs are specified base/quote and strategy is entered in quote per base units
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='fast', xf=True)

## Add strategies

In [57]:
for i in range(2):
    Sim.add_strategy(
        tkn=f'{tkn0}',          # specify sell side tkn
        amt_sell=20 * 10**token0Decimals,            # liqudity amount of tkn to sell
        psell_start=2000 * 10**delta_tokenDecimals,       # price to start selling in quote per base units
        psell_end=2500 * 10**delta_tokenDecimals,         # price to stop selling in quote per base units
        amt_buy=10000 * 10**token1Decimals,          # liqudity amount of paired tkn to buy
        pbuy_start=1700 * 10**delta_tokenDecimals,        # price to start buying in quote per base units
        pbuy_end=1000 * 10**delta_tokenDecimals,          # price to stop buying in quote per base units
        pair=f"{tkn0}/{tkn1}",   # pair
        # interpret_decimals= True,
        # tkn_decimals = token0Decimals,
        # tkn_other_decimals = token1Decimals,
    )

In [58]:
# for i in range(2):
#     Sim.add_strategy(
#         tkn=f'{tkn0}',          # specify sell side tkn
#         amt_sell=2_000_000_000,            # liqudity amount of tkn to sell
#         psell_start=4.7e-10,       # price to start selling in quote per base units
#         psell_end=5e-10,         # price to stop selling in quote per base units
#         amt_buy=1,          # liqudity amount of paired tkn to buy
#         pbuy_start=4.5e-10,        # price to start buying in quote per base units
#         pbuy_end=4e-10,          # price to stop buying in quote per base units
#         pair=f"{tkn0}/{tkn1}"   # pair
#     )['orders']

## Show all strategies

In [59]:
Sim.state()['orderuis']

{0: CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=20000.0, S=2360.679774997894, yint=2e+19, y=2e+19, id=0, linked=<1>),
 1: CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=3.1622776601683795e-05, S=9.608279654492807e-06, yint=10000000000.0, y=10000000000.0, id=1, linked=<0>),
 2: CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=20000.0, S=2360.679774997894, yint=2e+19, y=2e+19, id=2, linked=<3>),
 3: CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=3.1622776601683795e-05, S=9.608279654492807e-06, yint=10000000000.0, y=10000000000.0, id=3, linked=<2>)}

In [62]:
Sim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,2.000000e+19,2.000000e+19,ETH,False,2.000000e-09,2.500000e-09,2.000000e-09,USDC per ETH,1,
1,1,ETHUSDC,USDC,1.000000e+10,1.000000e+10,USDC,False,1.700000e-09,1.000000e-09,1.700000e-09,USDC per ETH,0,
2,2,ETHUSDC,ETH,2.000000e+19,2.000000e+19,ETH,False,2.000000e-09,2.500000e-09,2.000000e-09,USDC per ETH,3,
3,3,ETHUSDC,USDC,1.000000e+10,1.000000e+10,USDC,False,1.700000e-09,1.000000e-09,1.700000e-09,USDC per ETH,2,


In [61]:
order = Sim.state()['orderuis'][0]
order.pair.set_decimals(dct=tokenDecimals)
order.yzABS(sx=48)

yzABSdata(y=2e+37, z=2e+37, A=664472284688848940695552, B=5629499534213120000000000, S=281474976710656)

In [84]:
order = Sim.state()['orderuis'][0]
alphaxutils.encodeFloat(int(order.B*10**token0Decimals))

7748835983126188

In [80]:
EncodedOrder('ETH',20000000000000000000,
   20000000000000000000,
   3539924399406984,
   4393923342499840).A_

2360.6797749978723

In [81]:
EncodedOrder('ETH',20000000000000000000,
   20000000000000000000,
   3539924399406984,
   4393923342499840).B_

20000.0

In [ ]:
EncodedOrder('USDC',10000000000, 10000000000, 2704490292, 8901020307).A_

9.608279654571561e-06

In [49]:
EncodedOrder('USDC',10000000000, 10000000000, 2704490292, 8901020307).B_

3.1622776599959934e-05

In [31]:
EncodedOrder.encode(y=2e+19, z=2e+19, A=664472284688849664, B=5629499534213120000, S=281474976710656)

TypeError: EncodedOrder.encode() got an unexpected keyword argument 'A'

In [24]:
EncodedOrder.from_sdk(Sim.state()['orderuis'][0])

TypeError: EncodedOrder.from_sdk() missing 1 required positional argument: 'order'

In [ ]:
# int(Sim.orders[0].B *2**48)
int(Sim.orders[0].S *2**48/2)

In [ ]:
3539924399406984

In [28]:
for key in Sim.orders.keys():
    test_order = CarbonOrderUI.from_order(Sim.orders[key])
    test_order.pair.set_decimals(dct=tokenDecimals)
    print(key, test_order.yzABS(sx=48))

0 yzABSdata(y=2e+19, z=2e+19, A=664472284688849664, B=5629499534213120000, S=281474976710656)
1 yzABSdata(y=10000000000.0, z=10000000000.0, A=2704490291, B=8901020307, S=281474976710656)
2 yzABSdata(y=2e+19, z=2e+19, A=664472284688849664, B=5629499534213120000, S=281474976710656)
3 yzABSdata(y=10000000000.0, z=10000000000.0, A=2704490291, B=8901020307, S=281474976710656)


In [ ]:
break

In [ ]:
8901020307
8901020307

In [ ]:
3539924399406984
2595594862065819

## Do Trades in each direction
- Note that `execute=False` so each of these trades behave as if they are independent - i.e. the order balances do NOT change post-trade

## Enter an amount

In [ ]:
amount = 0.000005

## APP - "Buy ETH with USDC" box

In [ ]:
# Enter 1 USDC into the "BUY ETH for USDC" box
tbSou10 = Sim.amm_buys(f"{tkn1}", amount, execute=False, support_partial=False) # route_trade_by_source  #note the variable name "tbSou10" == {tradeBySource}{sourceToken}{targetToken}. e.g. in the TokensTraded event tkn1 should be the source token
tbSou10['trades']

In [ ]:
amount = 500000000

In [ ]:
# Enter 1 ETH into the "BUY ETH for USDC" box
tbTar10 = Sim.amm_sells(f"{tkn0}", amount, execute=False, support_partial=False) # route_trade_by_target
tbTar10['trades']

## APP - "Sell ETH with USDC" box

In [ ]:
amount = 1170038356

In [ ]:
# Enter 1 ETH into the "SELL ETH for USDC" box
tbSou01 = Sim.amm_buys(f"{tkn0}", amount, execute=False, support_partial=False) # route_trade_by_source
tbSou01['trades']

In [ ]:
# Enter 1 USDC into the "SELL ETH for USDC" box
tbTar01 = Sim.amm_sells(f"{tkn1}", amount, execute=False, support_partial=False) # route_trade_by_target
tbTar01['trades']

## Summary of trades

In [ ]:
Sim.state()["trades"].query("subid=='A'")